In [1]:
import pandas as pd
import numpy as np

In [9]:
# Data: 
xls_dict  = pd.read_excel('data/trading-game-data-08112023.xlsx', sheet_name=None)

info_df = xls_dict['info'][['Ticker', 'RBICS Economy']]
info_df = info_df.set_index('Ticker')

index_price_df = xls_dict['index-price']
price_df = xls_dict['price']
size_df = xls_dict['size']
price_to_book_df = xls_dict['price-to-book']
turnover_df = xls_dict['turnover']

price_df = xls_dict['price'].reset_index()
price_df = price_df.drop('index', axis = 1)
price_df['Date'] = pd.to_datetime(price_df['Date'])
price_df.set_index('Date', inplace=True)

In [34]:
def calculate_monthly_returns(price_df):
    # Assuming price_df is indexed by date and contains end-of-day prices
    monthly_prices = price_df.resample('M').last()  # Taking the last price of each month
    monthly_returns = np.log(monthly_prices / monthly_prices.shift(1))
    return monthly_returns


def calc_sentiment_outsample(price_df):
    log_returns = np.log(price_df / price_df.shift(1))

    n_obs = len(log_returns)
    # Calculate volatility (of the )
    volatility = log_returns.rolling(window=6*21).std() * np.sqrt(n_obs)
    volatility_mean = volatility.mean()

    # Step 2: Sort stocks and create buckets
    sorted_stocks = volatility_mean.sort_values()
    buckets = np.array_split(sorted_stocks, 10)  # Splitting into 10 buckets

    # Step 3: using the returns; 
    monthly_log_returns = calculate_monthly_returns(price_df)

    return buckets, monthly_log_returns
    

def calculate_future_returns(current_prices, future_prices):
    # Current prices are 
    return np.log(future_prices/current_prices)



buckets = calc_sentiment_outsample(price_df)


                   A       AAL      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2022-12-31       NaN       NaN       NaN       NaN       NaN       NaN   
2023-01-31  0.016107  0.238125  0.104829 -0.089664  0.262004  0.006898   
2023-02-28 -0.068791 -0.009963  0.021393  0.040781  0.103937 -0.083244   
2023-03-31 -0.025901 -0.080095  0.112121  0.034926  0.009044 -0.004532   
2023-04-30 -0.021260 -0.078236  0.028575 -0.053154 -0.038764  0.087053   
2023-05-31 -0.157714  0.080268  0.043647 -0.091110 -0.086351 -0.079771   
2023-06-30  0.038832  0.193758  0.090142 -0.023691  0.154892  0.066559   
2023-07-31  0.012561 -0.068635  0.012704  0.104559  0.171850  0.020967   
2023-08-31 -0.005765 -0.128512 -0.044658 -0.017671 -0.145743 -0.078741   
2023-09-30 -0.079478 -0.139660 -0.092859  0.014188  0.042126 -0.060594   
2023-10-31 -0.078576 -0.138787 -0.002573 -0.054313 -0.148373 -0.024035   
2023-11-30  0.067063  0.052414  0.0627